# note: do evaluation in (N,C,L) format, this way the accuracy is better reflected, and we still get stel predictions over time

# TODO: historic average and need a map of time step to historic step...
- ### The number of hours of the division should give an indication of cell plot scales 
- ### Separate model results and model metrics: allows for less disc space being used


### What will have more weight false positive or false negative? Obviously false positive is a lot more important. Intuitively put, recisions measures to how many of our class guesses did we waste, where as recall measures how many of the actual occurrences did we catch.

In [1]:
import os
import logging as log
from time import strftime
from copy import deepcopy
from torch import nn, optim
import torch.nn.functional as F
from utils.data_processing import *
from logger.logger import setup_logging
from utils.configs import BaseConf
from utils.utils import write_json, Timer
from models.kangkang_fnn_models import KangFeedForwardNetwork
from dataloaders.flat_loader import FlatDataLoaders
from datasets.flat_dataset import FlatDataGroup, BaseDataGroup
from utils.plots import DistributionPlotter
from utils.metrics import PRCurvePlotter, ROCCurvePlotter, LossPlotter, CellPlotter
from sklearn.metrics import accuracy_score, average_precision_score, roc_auc_score, classification_report

In [2]:
%matplotlib inline
from utils.mock_data import mock_data
import matplotlib.pyplot as plt
from utils.plots import im
from utils.metrics import best_threshold, get_y_pred
from models.model_result import ModelResult
from models.baseline_models import ExponentialMovingAverage,\
                UniformMovingAverage, TriangularMovingAverage, HistoricAverage

In [3]:
os.listdir("./data/processed")

['T6H-X850M-Y880M',
 'T24H-X85M-Y110M',
 'T24H-X850M-Y880M',
 'T24H-X425M-Y440M',
 'T12H-X850M-Y880M',
 'T1H-X1700M-Y1760M',
 'T3H-X850M-Y880M']

In [23]:
YEAR_OFFSET = 3
crime_grids = np.arange(20)
crime_type_grids = np.arange(20)
t_range = pd.date_range(
    start='2012',
    end='2015',
    periods=len(crime_grids),
)

last_year_crime_grids = np.copy(crime_grids[:-YEAR_OFFSET])
crime_grids = crime_grids[YEAR_OFFSET:]
crime_type_grids = crime_type_grids[YEAR_OFFSET:]
print(t_range)
t_range =  t_range[YEAR_OFFSET:]
print(t_range)
print(crime_grids)
print(last_year_crime_grids)

DatetimeIndex([          '2012-01-01 00:00:00',
               '2012-02-27 16:25:15.789473684',
               '2012-04-25 08:50:31.578947368',
               '2012-06-22 01:15:47.368421052',
               '2012-08-18 17:41:03.157894736',
               '2012-10-15 10:06:18.947368420',
               '2012-12-12 02:31:34.736842104',
               '2013-02-07 18:56:50.526315788',
               '2013-04-06 11:22:06.315789472',
               '2013-06-03 03:47:22.105263152',
               '2013-07-30 20:12:37.894736840',
               '2013-09-26 12:37:53.684210528',
               '2013-11-23 05:03:09.473684208',
               '2014-01-19 21:28:25.263157888',
               '2014-03-18 13:53:41.052631576',
               '2014-05-15 06:18:56.842105264',
               '2014-07-11 22:44:12.631578944',
               '2014-09-07 15:09:28.421052624',
               '2014-11-04 07:34:44.210526304',
                         '2015-01-01 00:00:00'],
              dtype='datetime64[ns]', f

In [4]:
# todo add to utils
def get_time_step_from_data_folder(folder_name):
    """
    Also used to get the window on the cell plots
    """
    i = folder_name.find("H")
    if i < 0:
        return None
    return int(folder_name[1:i])

def get_historic_step_from_time_step(time_step):
    if time_step:
        return int(24/time_step)

In [7]:
data_dim_str = "T24H"
get_historic_step_from_time_step(get_time_step_from_data_folder(data_dim_str))

1

In [ ]:
conf_dict = {
    "seed": 3,
    "use_cuda": False,
    
    "use_crime_types": False,
    
    # data group/data set related
    "val_ratio": 0.1,  # ratio of the total dataset
    "tst_ratio": 0.2,# ratio of the total dataset
    "seq_len": 1,
    "flatten_grid": True,  # if the shaper should be used to squeeze the data
    
    # shaper related 
    "shaper_top_k": -1,  # if less then 0, top_k will not be applied
    "shaper_threshold": 0,

    
    # data loader related
    "sub_sample_train_set": True,
    "sub_sample_validation_set": True,
    "sub_sample_test_set": False,
    
    # training parameters
    "resume": False,
    "early_stopping": False,
    "tolerance": 1e-8,
    "lr": 1e-3,
    "weight_decay": 1e-8,
    "max_epochs": 1,
    "batch_size": 64,
    "dropout": 0.2,
    "shuffle": False,
    "num_workers": 6,
}

conf = BaseConf(conf_dict)


# data_dim_str = 'T6H-X850M-Y880M'
# data_dim_str = 'T24H-X85M-Y110M'
data_dim_str = 'T24H-X850M-Y880M'
# data_dim_str = 'T24H-X425M-Y440M'
# data_dim_str = 'T12H-X850M-Y880M'
# data_dim_str = 'T3H-X850M-Y880M'
# data_dim_str = 'T1H-X1700M-Y1760M'

data_path = f"./data/processed/{data_dim_str}/"
os.makedirs(data_path, exist_ok=True)

for the model_results - just save targets and probas_pred - the predictions should be handled by a function,
for example getting the best_threshold per cell and then getting the y_pred - hard classifications.

this allows that the format of the results are in the correct order. Be bold, work constantly. Try new things.

In [ ]:
im(data=crimes[:,0].mean(0),figsize=(5,5))

In [ ]:
from utils.preprocessing import Shaper


with np.load(data_path + "generated_data.npz") as zip_file:
    t_range = pd.read_pickle(data_path + "t_range.pkl")
    log.info(f"\tt_range shape {np.shape(t_range)}")

    if conf.use_crime_types:
        crimes = zip_file["crime_types_grids"]
    else:
        crimes = zip_file["crime_grids"]

conf.shaper_threshold = 0
conf.shaper_top_k = 100

shaper = Shaper(data=crimes,
                threshold=conf.shaper_threshold,
                top_k=conf.shaper_top_k)

crimes_ = shaper.squeeze(crimes)
crimes_.shape


In [ ]:
conf.shaper_threshold = 0
conf.shaper_top_k = -1
data_group = FlatDataGroup(data_path=data_path, conf=conf)

In [ ]:
# visualize the cells that we are dropping
conf.shaper_threshold = 0
conf.shaper_top_k = -1

for i in [-1]:
    conf.shaper_top_k = i
    data_group = FlatDataGroup(data_path=data_path, conf=conf)
    print(f"threshold = {i}, {data_group.targets.shape}")
    vals, counts = np.unique(data_group.targets,return_counts=True)
    counts = counts/np.sum(counts)
    dist = dict(zip(vals,counts))
    print(dist)
    
    sorted_indices = data_group.sorted_indices
    
    plt.figure(figsize=(5,5))
    plt.imshow(data_group.shaper.unsqueeze(data_group.crimes)[:,0].mean(0))
    plt.colorbar()
    plt.show()

# SKLearn Dummy Classifier

In [ ]:
from sklearn.dummy import DummyClassifier
from sklearn.linear_model import LogisticRegression

dummy = DummyClassifier(strategy='prior')
dummy.fit(X,y)

lr = LogisticRegression(solver='lbfgs')
lr.fit(X,y)

print("log_reg:",accuracy_score(y, lr.predict_proba(X)[:,1].round()))

for strat in ['stratified', 'most_frequent', 'prior', 'uniform']:
    dummy = DummyClassifier(strategy=strat)
    dummy.fit(X,y)
    print(f"dummy ({strat}):",accuracy_score(y, dummy.predict_proba(X)[:,1].round()))


# Logistic Regression Model

In [ ]:
X.shape, y.shape
y

In [ ]:
get mean model as the starting position.
then train with only the new values to see the difference

In [ ]:
data_group.crimes[:,0].shape

In [ ]:
from sklearn.datasets import load_iris
from sklearn.linear_model import LogisticRegression
X, y = load_iris(return_X_y=True)
clf = LogisticRegression(random_state=0, solver='lbfgs',
                         multi_class='multinomial')

clf..fit(X, y)
clf.predict(X[:2, :])
clf.predict_proba(X[:2, :]) 
clf.score(X, y)

In [ ]:
clf.coef_

In [ ]:
plt.figure(figsize=(15,15))
X = data_group.crimes[:400,0,data_group.sorted_indices]
aspect = X.shape[1]/X.shape[0]

plt.imshow(X=X,aspect=aspect,cmap='viridis')
plt.show()

In [ ]:
model_results = []

Hawkes models incorporate the mean and the self exciting variables - when we train each cell on the data and 
l(k) = u + wet-t-1 - save the parameters in a grid and plot. Should show us how much the thing is influence.
Espetially the ratio between the average and the actual excitation.


#### Historic average model

In [ ]:
time_step = get_historic_step_from_time_step(get_time_step_from_data_folder(data_dim_str))
print(f"using time step: {time_step}")

ha = HistoricAverage(step=time_step)
all_crimes = data_group.crimes[:,0]
all_targets = data_group.targets
tst_targets = data_group.testing_set.targets
all_crimes_ha = ha(all_crimes)  # todo re-write using the convolve function - it loses the 
tst_crimes_ha = all_crimes_ha[-len(tst_targets):]
trn_crimes_ha = all_crimes_ha[time_step+1:len(tst_targets)] # skip all the nan values
trn_targets = all_targets[time_step+1:len(tst_targets)]



N,L = tst_targets.shape


y_true = tst_targets
probas_pred = tst_crimes_ha
thresh = best_threshold(y_true, probas_pred)
y_pred = get_y_pred(thresh, probas_pred)  # might want to do this for each cell either?

# alternatively - might want to do this for each cell either?
# thresholds_ha = []
# y_pred = np.empty_like(y_true)
# for i in range(L):
#     # important note thresh should eb for the training data
#     thresh = best_threshold(y_true=trn_targets[:,i],probas_pred=trn_crimes_ha[:,i], verbose=False)  
#     thresholds_ha.append(thresh)
#     y_pred[:,i] = get_y_pred(thresh, probas_pred[:,i])
    
ha_model_result = ModelResult(model_name=f"HA One Thresh {time_step}",
                                y_true=y_true.flatten(),
                                y_pred=y_pred.flatten(),
                                probas_pred=probas_pred.flatten(),
                                t_range=data_group.testing_set.t_range,
                                shaper=data_group.shaper)

model_results.append(ha_model_result)

#### Mean of training data as future prediction

In [ ]:
# Mean of training data as future prediction
trn_crimes = data_group.training_set.crimes

# only get the mean of the trn_set
crimes_mean = np.mean(trn_crimes[:,0],axis=0,keepdims=True)  # keep dims used to make scalar product easy
crimes_ones = np.ones_like(data_group.testing_set.targets)
y_pred_sparse = crimes_mean*crimes_ones
tst_targets = data_group.testing_set.targets

N,L = tst_targets.shape
targets_shape = N,L


y_pred_dense = y_pred_sparse

probas_pred = y_pred_dense.flatten()
y_true = tst_targets.flatten()
thresh = best_threshold(y_true, probas_pred)
y_pred = np.copy(probas_pred)
y_pred[y_pred >= thresh] = 1
y_pred[y_pred < thresh] = 0

mean_model_result = ModelResult(model_name="Train Mean",
                                y_true=y_true,
                                y_pred=y_pred,
                                probas_pred=probas_pred,
                                t_range=data_group.testing_set.t_range,
                                shaper=data_group.shaper)

model_results.append(mean_model_result)

#### Rolling mean of all data 

In [ ]:
for window_len in [time_step*7]:
    for alpha in [1e1, 1e2,1e-1]:
        ma = ExponentialMovingAverage(alpha=alpha,window_len=window_len)
        all_crimes = data_group.crimes[:,0]
        tst_targets = data_group.testing_set.targets
        all_crimes_ma = ma(all_crimes)
        tst_crimes_ma = all_crimes_ma[-len(tst_targets):]

        N,L = tst_targets.shape
        targets_shape = N,L

        targets_dense = tst_targets

        y_pred_dense = tst_crimes_ma

        probas_pred = y_pred_dense.flatten()
        y_true = targets_dense.flatten()

        thresh = best_threshold(y_true, probas_pred)
        y_pred = np.copy(probas_pred)
        y_pred[y_pred >= thresh] = 1
        y_pred[y_pred < thresh] = 0

        ma_model_result = ModelResult(model_name=f"EMA window={window_len}, alpha={alpha}",
                                        y_true=y_true,
                                        y_pred=y_pred,
                                        probas_pred=probas_pred,
                                        t_range=data_group.testing_set.t_range,
                                        shaper=data_group.shaper)

        model_results.append(ma_model_result)

#### Adding random noise to the predictions

In [ ]:
for noise_std in [0.4,0.8]:
    tst_crimes = data_group.testing_set.targets  # only mask the targets as the outputs


    noise = noise_std*np.random.randn(*np.shape(tst_crimes))
    # should be a flip seeing that the class distribution is so skew

    y_pred_sparse = tst_crimes + noise
    targets_dense = data_group.testing_set.targets
    y_pred_dense = y_pred_sparse

    probas_pred = y_pred_dense.flatten()
    
    N,L = targets_dense.shape
    targets_shape = N,L
    
    y_true = targets_dense.flatten()
    thresh = best_threshold(y_true, probas_pred)
    y_pred = np.copy(probas_pred)
    y_pred[y_pred >= thresh] = 1
    y_pred[y_pred < thresh] = 0

    noise_model_result = ModelResult(model_name=f"Noise model std={noise_std}",
                                    y_true=y_true,
                                    y_pred=y_pred,
                                    probas_pred=probas_pred,
                                    t_range=data_group.testing_set.t_range,
                                     shaper=data_group.shaper)

    model_results.append(noise_model_result)

In [ ]:
for result in model_results:
    print(result)

pr_plotter = PRCurvePlotter()
for result in model_results:
    pr_plotter.add_curve(result.y_true, result.probas_pred, label_name=result.model_name)
pr_plotter.show()
# pr_plotter.savefig(model_path + "plot_pr_curve.png")

roc_plotter = ROCCurvePlotter()
for result in model_results:
    roc_plotter.add_curve(result.y_true, result.probas_pred, label_name=result.model_name)
roc_plotter.show()
# roc_plotter.savefig(model_path + "plot_roc_curve.png")

In [ ]:
# todo create function do unflatten y_true, y_probs, and probas_true
# use shape of the model and just rnp.reshape


### [Hawkes Model](https://x-datainitiative.github.io/tick/modules/generated/tick.hawkes.HawkesSumExpKern.html)
does support multi variate but we are only using univariate for now

### Data Analysis Hawks-proces using the EM kernel

In [ ]:
# todo get average using convolutions given the kernel 

In [ ]:
from tick.plot import plot_hawkes_kernels
from tick.hawkes import SimuHawkesSumExpKernels, SimuHawkesMulti, \
    HawkesSumExpKern, HawkesEM


class IndHawkesModel:
    """
    Indipendent Hawkes Modles where all cells are indipendent
    
    Using Tikc library from:
    - https://x-datainitiative.github.io/tick/modules/generated/tick.hawkes.HawkesSumExpKern.html
    """

    def __init__(self, kernel_size):
        self.kernel_size = kernel_size
        self.baselines = []   
        self.kernels = []

    def fit(self, data):
        """
        data: (N,L)
        """
        N, L = data.shape
        self.baselines = []
        self.kernels = []

        # convert into format for the hawkes trainer
        for i in range(L):
            realizations = []
            data_counts = data[:,i]
            time_stamps = np.argwhere(data_counts).astype(np.float)
            realizations.append([time_stamps[:,0]])

            # kernel_discretization if set explicitly it overrides kernel_support and kernel_size
            # todo: have kernel values be set by conf_dict
            em = HawkesEM(kernel_discretization=np.arange(self.kernel_size).astype(np.float),
                          n_threads=8,
                          verbose=False,
                          tol=1e-3,
                          max_iter=1000)
            em.fit(realizations)
            baseline = em.baseline.flatten()[0]
            kernel = em.kernel.flatten()
            self.baselines.append(baseline)
            self.kernels.append(kernel)

        
    def transform(self, data):
        # todo consider training saving a kernel AND baseline for each cell
        # kernels -> (N,L,kernel_size)
        # baselines -> (N,L,1)
        N,L = np.shape(data)
        result = np.empty_like(data)
        for i in range(L):    
            result[:,i] = self.baselines[i] + np.convolve(data[:,i],self.kernels[i])[:N]
        return result
    
    def fit_transform(self,data):
        self.fit(data)
        return self.transform(data)
    

In [ ]:
time_step

In [ ]:
trn_inpt = data_group.training_set.crimes[:,0]
trn_trg = data_group.training_set.targets

tst_inpt = data_group.testing_set.crimes[:,0]
tst_trg = data_group.testing_set.targets


N,L = np.shape(trn_inpt)

model = IndHawkesModel(kernel_size=time_step*21 + 1)
trn_out = model.fit_transform(trn_inpt)
tst_out = model.transform(tst_inpt)

y_true = tst_trg
probas_pred = tst_out
thresh = best_threshold(y_true, probas_pred)
y_pred = get_y_pred(thresh, probas_pred)  # might want to do this for each cell either?

hawkes_ind_model = ModelResult(model_name=f"Hawkes Independent Cells",
                                y_true=y_true.flatten(),
                                y_pred=y_pred.flatten(),
                                probas_pred=probas_pred.flatten(),
                                t_range=data_group.testing_set.t_range,
                               shaper=data_group.shaper)

model_results.append(hawkes_ind_model)

In [ ]:
kernels = np.array(model.kernels)
kernel_mean = np.mean(kernels,0)
kernel_std = np.std(kernels,0)
plt.plot(kernel_mean,alpha=.4)

x = np.concatenate([np.arange(len(kernel_mean)),np.arange(len(kernel_mean))[::-1]])
y = np.concatenate([kernel_mean-1.5*kernel_std,np.array(kernel_mean+1.5*kernel_std)[::-1]])
plt.fill(x,y,alpha=0.1)
plt.show()

In [ ]:
class HawkesModelGeneral:
    """
    Using the HawkesEM learner from Tick library
    
    Using Tikc library from:
    - https://x-datainitiative.github.io/tick/modules/generated/tick.hawkes.HawkesSumExpKern.html
    """

    def __init__(self, kernel_size):
        self.kernel_size = kernel_size
        self.kernel = None
        self.baseline = None
        self.em = None  

    def fit(self, data):
        """
        data: (N,L)
        """
        N, L = data.shape

        # convert into format for the hawkes trainer
        realizations = []
        for i in range(L):
            data_counts = data[:,i]
            time_stamps = np.argwhere(data_counts).astype(np.float)
        realizations.append([time_stamps[:,0]])

        # kernel_discretization if set explicitly it overrides kernel_support and kernel_size
        # todo: have kernel values be set by conf_dict
        self.em = HawkesEM(kernel_discretization=np.arange(self.kernel_size).astype(np.float),
                      n_threads=8,
                      verbose=False,
                      tol=1e-3,
                      max_iter=1000)
        self.em.fit(realizations)
        self.baseline = self.em.baseline.flatten()[0]
        self.kernel = self.em.kernel.flatten()

        
    def transform(self, data):
        # todo consider training saving a kernel AND baseline for each cell
        # kernels -> (N,L,kernel_size)
        # baselines -> (N,L,1)
        N,L = np.shape(data)
        result = np.empty_like(data)
        for i in range(L):    
            result[:,i] = self.baseline + np.convolve(data[:,i],self.kernel)[:N]
        return result
    
    def fit_transform(self,data):
        self.fit(data)
        return self.transform(data)
    
    def plot_kernel(self):
        plot_hawkes_kernels(self.em, hawkes=None, show=True)    

In [ ]:
trn_inpt = data_group.training_set.crimes[:,0]
trn_trg = data_group.training_set.targets

tst_inpt = data_group.testing_set.crimes[:,0]
tst_trg = data_group.testing_set.targets


N,L = np.shape(trn_inpt)

model = HawkesModelGeneral(kernel_size=time_step*20 + 1)
trn_out = model.fit_transform(trn_inpt)
tst_out = model.transform(tst_inpt)

y_true = tst_trg
probas_pred = tst_out
thresh = best_threshold(y_true, probas_pred)
y_pred = get_y_pred(thresh, probas_pred)  # might want to do this for each cell either?

hawkes_general_model = ModelResult(model_name=f"Hawkes General Model",
                                y_true=y_true.flatten(),
                                y_pred=y_pred.flatten(),
                                probas_pred=probas_pred.flatten(),
                                t_range=data_group.testing_set.t_range,
                                   shaper=data_group.shaper)

model_results.append(hawkes_general_model)

plt.plot(model.kernel)

In [ ]:
# try and get cosine similarity between the kernels and baselines - should give a good indicaiton if we can compress

### ISSUE: is that we have some cells where no crime takes place - in either the testing or the training set
### thougths - look for data where to ranking of the cells actually changes with time - this will indicate how much crime actaully moves arround over time

In [ ]:
trn_inpt = data_group.training_set.crimes[:,0]
trn_trg = data_group.training_set.targets

tst_inpt = data_group.testing_set.crimes[:,0]
tst_trg = data_group.testing_set.targets


N,L = np.shape(trn_inpt)

model = IndHawkesModel(kernel_size=time_step*3 + 1)
trn_out = model.fit_transform(trn_inpt)
tst_out = model.transform(tst_inpt)

limit = 200
top_k = 10
i = 3000


# todo check how this influences the ROC and PR curves
def i2p(intensity):
    """
    intensity to probability
    """
    return 1 - np.exp(-1*intensity)
    
       
for i in range(top_k):
    for j in range(2):
        print(f"-------------------------------{j}----------------------------------")
        plt.figure(figsize=(10,2))
        plt.plot(tst_trg[i:i+limit,i])

        y_true = tst_trg[:,i]
        if j == 0:
            probas_pred = i2p(tst_out[:,i])
        else:     
            probas_pred = tst_out[:,i]

        thresh = best_threshold(y_true, probas_pred)
        y_pred = np.copy(probas_pred)
        y_pred[y_pred >= thresh] = 1
        y_pred[y_pred < thresh] = 0

        print(classification_report(y_true=y_true,y_pred=y_pred))

        plt.plot(y_pred[i:i+limit])
        plt.plot(probas_pred[i:i+limit])
    #     plt.ylim([0,1])
        plt.show()

        print(f"------------------------------------------------------------------")

        

In [ ]:
plt.plot(np.linspace(0,4,100))
plt.plot(i2p(np.linspace(0,4,100)))

In [ ]:
from sklearn.metrics import accuracy_score, average_precision_score, roc_auc_score, classification_report

In [ ]:
for realization in realizations[:5]:#multi.timestamps:
    plt.figure(figsize=(10,1))
    for var in realization:
        plt.scatter(var, np.ones_like(var), alpha=.2, marker="|")    
    plt.show()